In [ ]:
pip install pytesseract easyocr paddleocr pillow torch torchvision transformers doctr[torch]


In [ ]:
from PIL import Image
import pytesseract
import easyocr
from paddleocr import PaddleOCR
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import torch


image_path = "sample.jpg"   # <-- put your test image here
img = Image.open(image_path)


print("\n==================== 1. TESSERACT ====================")
print(pytesseract.image_to_string(img))



print("\n==================== 2. EASYOCR ====================")
reader = easyocr.Reader(['en'])
print(reader.readtext(image_path, detail=0))



print("\n==================== 3. PADDLE OCR ====================")
ocr = PaddleOCR(lang='en')
result = ocr.ocr(image_path)
for r in result:
    print(r[1][0])



print("\n==================== 4. TrOCR (HuggingFace) ====================")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-printed")

pixel_values = processor(images=img, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(text)



print("\n==================== 5. DocTR ====================")
model = ocr_predictor(pretrained=True)
doc = DocumentFile.from_images(image_path)
result = model(doc)
print(result.render()[0])
